### 2. AI 챗봇 - Flask 서버 및 db 대화 규칙 그리고 챗봇 실행

In [1]:
# 필요한 것 임포트

from flask import Flask, request, jsonify
from flask_cors import CORS
import sqlite3
import nltk
from nltk.chat.util import Chat, reflections
import json

In [2]:
# NLTK 다운로드 
# NLTK는 일부 말뭉치와 모델을 별도로 다운로드해야 하므로, punkt 패키지가 없으면 토큰화 함수들이 제대로 작동하지 않습니다
nltk.download('punkt') 

app = Flask (__name__)
CORS(app)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# 데이터베이스에서 대화 규칙 불러오기
def load_auraystudio_from_db():
    conn = sqlite3.connect('auraystudio.db')
    cursor = conn.cursor()
    cursor.execute('SELECT pattern, response FROM chatbot_rules')
    rows = cursor.fetchall()
    conn.close()

    pairs = []
    for pattern, response in rows:
        response_list = json.loads(response)  # JSON 형식으로 저장된 응답을 리스트로 변환
        pairs.append([pattern, response_list])
    return pairs

In [4]:
# DB에서 규칙 불러오기
pairs = load_auraystudio_from_db()

In [5]:
# 챗봇 인스턴스 생성
chatbot = Chat(pairs, reflections)

In [6]:
@app.route ('/chatbot', methods=['POST'])
def chatbot_response () :
    try:
        user_message = request.json.get('message')
        bot_response = chatbot.respond(user_message)
        return jsonify({'response' : bot_response})
    except Exception as e :
        return jsonify({'error' : str(e)}), 500

# 아래처럼 적으면 localhost주소로만 넘어가게 되어 네트워크로는 안돌아가기 때문에
#  (host='0.0.0.0', port=5000) 모든 ip에서 접근가능하게 설정할 수 있는 코드

# 1번째 : 모든 ip에서 접근 불가능하게 하고 싶을 때
#if __name__ == '__main__' :
#    app.run(port=5000)

# 2번째 : 모든 ip에서 접근 가능하게 하고 싶을 때
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.17:5000
Press CTRL+C to quit
192.168.1.17 - - [30/Sep/2024 10:06:05] "OPTIONS /chatbot HTTP/1.1" 200 -
192.168.1.17 - - [30/Sep/2024 10:06:05] "POST /chatbot HTTP/1.1" 200 -
